<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/FCN_Experiments/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/FCN_Experiments/'

In [8]:
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 0 --opt HMC --gpu 0 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt HMC --gpu 1 &

dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'HMC', 'gpu': '0'}

rob = dict['rob']
opt = dict['opt']
inference = opt

In [9]:
import BayesKeras
from BayesKeras import PosteriorModel
from BayesKeras import analyzers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)


model = PosteriorModel(experiment + "%s_FCN_Posterior_%s"%(inference, rob))

[INFO] BayesKeras: Attempting to load a sample based posterior
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, None, None, 512)   401920    
                                                                 
 dense_7 (Dense)             (None, None, None, 10)    5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


In [10]:

from tqdm import trange

loss = tf.keras.losses.SparseCategoricalCrossentropy()

num_images = 500

accuracy = tf.keras.metrics.Accuracy()
preds = model.predict(X_test[0:500]) #np.argmax(model.predict(np.asarray(adv)), axis=1)
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:500])
fgsm = accuracy.result()
print("%s Accuracy: "%(inference), accuracy.result())

accuracy = tf.keras.metrics.Accuracy()
adv = analyzers.PGD(model, X_test[0:500], eps=0.1, loss_fn=loss, num_models=10)
preds = model.predict(adv) #np.argmax(model.predict(np.asarray(adv)), axis=1)
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:500])
fgsm = accuracy.result()
print("FGSM Robustness: ", accuracy.result())

accuracy = tf.keras.metrics.Accuracy()
preds = analyzers.chernoff_bound_verification(model, X_test[0:100], 0.1, y_test[0:100], confidence=0.80)
#print(preds.shape)
#print(np.argmax(preds, axis=1).shape)
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:100])
print("Chernoff Lower Bound (IBP): ",  accuracy.result())

"""
p = 0
for i in trange(100, desc="Computing FGSM Robustness"):
    this_p = analyzers.massart_bound_check(model, np.asarray([X_test[i]]), 0.075, y_test[i])
    print(this_p)
    p += this_p
print("Massart Lower Bound (IBP): ", p/100.0)
"""


HMC Accuracy:  tf.Tensor(0.918, shape=(), dtype=float32)
FGSM Robustness:  tf.Tensor(0.768, shape=(), dtype=float32)


Sampling for Chernoff Bound Satisfaction: 100%|██████████| 24/24 [00:00<00:00, 54.24it/s]

Chernoff Lower Bound (IBP):  tf.Tensor(0.0, shape=(), dtype=float32)


'\np = 0\nfor i in trange(100, desc="Computing FGSM Robustness"):\n    this_p = analyzers.massart_bound_check(model, np.asarray([X_test[i]]), 0.075, y_test[i])\n    print(this_p)\n    p += this_p\nprint("Massart Lower Bound (IBP): ", p/100.0)\n'